In [1]:
import numpy as np
import pandas as pd
import sklearn.metrics as mtr
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 150)

In [2]:
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import Callback, EarlyStopping
from keras import backend as K

Using TensorFlow backend.


# Load Data

In [3]:
X = pd.read_pickle("X")

In [4]:
y = pd.read_pickle("y")

# Train Model

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [6]:
def crps(y_true, y_pred):
    y_true = K.cumsum(y_true, axis = 1)
    y_pred = K.cumsum(y_pred, axis = 1)
    return K.mean(K.mean(K.square(y_true - y_pred), axis = 1), axis = 0)

In [7]:
model = Sequential()
model.add(Dense(512, input_dim=X.shape[1], activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(199, activation='softmax'))

model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics = [crps])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               12800     
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 199)               51143     
Total params: 195,271
Trainable params: 195,271
Non-trainable params: 0
_________________________________________________________________


In [8]:
es = EarlyStopping(monitor='val_crps', 
                   mode='min',
                   patience = 3,
                   restore_best_weights = True)

In [176]:
class Metric(Callback):
    def __init__(self):
        super().__init__()
        self.losses = pd.DataFrame()
    def on_epoch_end(self, epoch, logs={}):
        self.losses = self.losses.append(logs, ignore_index = True)

In [177]:
metric = Metric()

In [178]:
fit = model.fit(X_train,
                y_train,
                epochs=10,
                batch_size = 1024,
                validation_data=[X_test, y_test],
                callbacks = [metric, es],
                verbose = 1)

Train on 18536 samples, validate on 4635 samples
Epoch 1/10
18536/18536 [==============================] - 0s 16us/step - loss: 1.9462 - crps: 0.0104 - val_loss: 3.8530 - val_crps: 0.0157
Epoch 2/10
18536/18536 [==============================] - 0s 15us/step - loss: 1.9487 - crps: 0.0106 - val_loss: 3.8714 - val_crps: 0.0159
Epoch 3/10
18536/18536 [==============================] - 0s 15us/step - loss: 1.9366 - crps: 0.0105 - val_loss: 3.8653 - val_crps: 0.0158
Epoch 4/10
18536/18536 [==============================] - 0s 15us/step - loss: 1.9470 - crps: 0.0106 - val_loss: 3.8689 - val_crps: 0.0158


In [179]:
metric.losses

,crps,loss,val_crps,val_loss
0,0.010423,1.946210,0.015665,3.852967
1,0.010586,1.948681,0.015904,3.871399
2,0.010456,1.936626,0.015823,3.865349
3,0.010567,1.946974,0.015783,3.868883
